In [1]:
import polars as pl
import pandas as pd
import pyodbc
from datetime import date

import time
start_time = time.time()

In [2]:
from IPython import get_ipython
get_ipython().run_line_magic("run", "01_pagos_recup_functions.ipynb")

#### Definición de Rutas

In [3]:
cierre = "202505"
cierre_l = "May25"
cierre_d = date(2025,5,31)

# Carpetas
path = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = path + "data/raw/"
wd_data_external = path + "data/external/"

wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/DWH/"
wd_data_validations = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/validations/"

# Archivos
## Inputs
fl_desembolsos_p1 = wd_data_raw + f'Desembolsos_P1_{cierre}'
fl_desembolsos_p1_bmxt = wd_data_raw + f'Desembolsos_P1_80686_{cierre}'
fl_desembolsos_p1_fianzas = wd_data_raw + f'Desembolsos_P1_BMXT_{cierre}'
fl_desembolsos_p2 = wd_data_raw + f'Desembolsos_P2_{cierre}'
fl_desembolsos_p2_bmxt = wd_data_raw + f'Desembolsos_P2_80686_{cierre}'
fl_desembolsos_p2_fianzas = wd_data_raw + f'Desembolsos_P2_BMXT_{cierre}'
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_recuperaciones = wd_data_raw + f'Recuperaciones_{cierre}.csv'
fl_recuperaciones_bmxt = wd_data_raw + f'Recuperaciones_BMXT_{cierre}.mdb'

# Outputs
## Tablas 
fl_pagadas_global_vf = wd_data_processed_dwh + f"parquet/Pagadas_Global_VF_{cierre}.parquet"
fl_recuperadas_global_vf = wd_data_processed_dwh + f'parquet/Recuperadas_Global_VF_{cierre}.parquet'
fl_recupera_con_pagos_flujos = wd_data_processed_dwh + f'parquet/Recupera_con_Pagos_Flujos_{cierre}.parquet'

## Validadores
fl_valida_pagos = wd_data_validations + f'Valida_Pagos.csv'
fl_valida_dwh_dac = wd_data_validations + f'Recuperadas_Valida_DWHvsDAC.csv'
fl_valida_td = wd_data_validations + f'Recuperadas_Valida_TD.csv'
fl_valida_base_pagos = wd_data_validations + f'Valida_Base_Pagos.csv'
fl_valida_base_pagos_mn = wd_data_validations + f'Valida_Base_Pagos_Mn.csv'
fl_valida_base_recup = wd_data_validations + f'Valida_Base_Recup.csv'
fl_valida_base_recup_mn = wd_data_validations + f'Valida_Base_Recup_Mn.csv'

# Outputs Access
acc_pagadas_global_vf = wd_data_processed_dwh + f"Pagadas_Global_VF_{cierre}.accdb"
acc_querie_pagadas = wd_data_processed_dwh + f"Querie_Pagadas_{cierre}.accdb"
acc_querie_recuperaciones = wd_data_processed_dwh + f"Querie_Recuperaciones_{cierre}.accdb"
acc_querie_unionflujos = wd_data_processed_dwh + f"Querie_UnionFlujos_{cierre}.accdb"
acc_recupera_con_pagos_flujos = wd_data_processed_dwh + f"Querie_Recupera_con_Pagos_Flujos_{cierre}.accdb"




### Catálogos

In [4]:
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)

# Solo Recuperaciones
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)

# Recuperaciones, Pagos
tipo_cambio = pl.read_database(query='SELECT * FROM [TIPO CAMBIO]', connection=conn)
programa = pl.read_database(query='SELECT * FROM [PROGRAMA]', connection=conn, schema_overrides={'PROGRAMA_ID': pl.Int128})
agrupamiento = pl.read_database(query='SELECT * FROM [AGRUPAMIENTO]', connection=conn)
udis = pl.read_database(query='SELECT * FROM [UDIS]', connection=conn, schema_overrides={'Fecha_Paridad': pl.Datetime})
#udis = udis.with_columns(pl.col("Fecha_Paridad").cast(pl.Date))
tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})
tipo_garantia = pl.read_database(query='SELECT * FROM [TIPO_GARANTIA]', connection=conn, schema_overrides={'Tipo_garantia_ID': pl.Int32})
sfc = pl.read_database(query='SELECT * FROM [SIN FONDOS CONTRAGARANTIA]', connection=conn)

conn.close()

tipo_cambio.columns = ["Anio", "Mes", "TC"]
tdc = tipo_cambio.filter((pl.col("Anio")==cierre_d.year) & (pl.col("Mes")==cierre_d.month)).select("TC").item()
tdc

19.3957

### Querie Pagos

In [5]:
def main_querie_pagadas():
    pagos_f1 = import_f1()
    pagos_f2 = import_f2()

    # Crea Concatenados y TPRO_CLAVE
    pagos_f1 = genera_concatenado(pagos_f1)
    pagos_f1 = genera_tpro_clave(pagos_f1)
    pagos_f2 = genera_concatenado(pagos_f2)

    pagadas_global_inter = genera_pagadas_global_inter(pagos_f1, pagos_f2)
    pagadas_global_vf = genera_pagadas_global_vf(pagadas_global_inter)
    pagadas_global_vf = complementa_pagadas_global_vf(pagadas_global_vf)

    valida_pagos = genera_valida_pagos(pagos_f2)
    valida_base_pagos = genera_valida_base_pagos(pagadas_global_vf)
    valida_base_pagos_mn = genera_valida_base_pagos_mn(pagadas_global_vf)

    # Exporta Tablas
    # pagadas_global_vf.write_csv(fl_pagadas_global_vf, datetime_format="%Y-%m-%d", float_precision=10)
    pagadas_global_vf.write_parquet(fl_pagadas_global_vf)

    # Exporta validadores
    valida_pagos.write_csv(fl_valida_pagos)
    valida_base_pagos.write_csv(fl_valida_base_pagos, float_precision = 10)
    valida_base_pagos_mn.write_csv(fl_valida_base_pagos_mn, float_precision = 10)
    

    return pagadas_global_vf

pagadas_global_vf = main_querie_pagadas()

In [6]:
pagadas_global_vf.filter(pl.col("Moneda_Id")==54)

Concatenado,Fecha_Consulta,Intermediario_Id,Numero_Credito,Producto,Pago ID,Razón Social (Intermediario),MIN_Fecha_Registro,Fecha_Garantia_Honrada,TPRO_CLAVE,Programa_Original,Programa_Id,Monto _Credito_Mn,Moneda_Id,Fecha de Apertura,Tipo_Garantia_Id,TIPO_PERSONA,RFC Empresa / Acreditado,Monto_Desembolsado,Interes_Desembolso,Interes_Moratorios,Porcentaje_Garantizado,Tipo_Credito_Id,Estatus_Recuperacion,Empresa / Acreditado (Descripción),Fecha Registro Alta,TC,AGRUPAMIENTO_ID,AGRUPAMIENTO,ESQUEMA,SUBESQUEMA,CAMBIO,MM_UDIS,NR_R,CSG,CSF,Monto_Desembolso_Mn,Interes_Desembolso_Mn,Interes_Moratorios_Mn,Monto_Pagado_Mn
str,datetime[μs],str,str,str,i64,str,datetime[μs],datetime[μs],i32,i32,i32,f64,i16,datetime[μs],i16,str,str,f64,f64,f64,f32,i16,str,str,datetime[μs],f64,f64,str,str,str,f64,i32,str,str,str,f64,f64,f64,f64
"""100400723198941""",2025-05-31 00:00:00,"""10040072""","""3198941""","""GARANTIA EMPRESARIAL""",1,"""BANCO MERCANTIL DEL NORTE""",null,2002-08-29 00:00:00,3003,3999,3003,362445.9732,54,2000-09-28 00:00:00,1,"""M""","""RME-981118-9F1""",-8462.03,0.0,null,50.0,6,"""RT""","""RUIBALAS DE MEXICO SA CV""",2001-01-30 12:34:07,19.3957,1275.0,"""Crediactivo Diana""","""PP""","""Tradicional""",2.84175,0,"""NR""","""CG""","""CF""",164126.995271,-0.0,null,164126.995271
"""100400726585030""",2025-05-31 00:00:00,"""10040072""","""6585030""","""GARANTIA EMPRESARIAL""",1,"""BANCO MERCANTIL DEL NORTE""",null,2003-04-30 00:00:00,3003,3999,3003,188004.1,54,2002-09-04 00:00:00,1,"""M""","""CHI-990710-132""",-7250.0,-58.09,null,50.0,7,"""RT""","""COMERCIALIZADORA HISA SA DE CV""",2002-10-11 13:28:25,19.3957,1275.0,"""Crediactivo Diana""","""PP""","""Tradicional""",3.15491,0,"""R""","""CG""","""CF""",140618.825,1126.696213,null,141745.521213
"""100400726621503""",2025-05-31 00:00:00,"""10040072""","""6621503""","""GARANTIA EMPRESARIAL""",1,"""BANCO MERCANTIL DEL NORTE""",null,2003-04-30 00:00:00,3003,3999,3003,359800.95,54,2002-09-12 00:00:00,1,"""M""","""CHI-990710-132""",-13875.0,-109.77,null,50.0,7,"""RT""","""COMERCIALIZADORA HISA SA DE CV""",2002-10-11 13:28:25,19.3957,1275.0,"""Crediactivo Diana""","""PP""","""Tradicional""",3.15832,0,"""R""","""CG""","""CF""",269115.3375,2129.065989,null,271244.403489
"""100400726642341""",2025-05-31 00:00:00,"""10040072""","""6642341""","""GARANTIA EMPRESARIAL""",1,"""BANCO MERCANTIL DEL NORTE""",null,2003-04-30 00:00:00,3003,3999,3003,489458.95,54,2002-09-18 00:00:00,1,"""M""","""CHI-990710-132""",-18875.0,-148.06,null,50.0,7,"""RT""","""COMERCIALIZADORA HISA SA DE CV""",2002-10-11 13:28:25,19.3957,1275.0,"""Crediactivo Diana""","""PP""","""Tradicional""",3.15979,0,"""R""","""CG""","""CF""",366093.8375,2871.727342,null,368965.564842
"""100400728557430""",2025-05-31 00:00:00,"""10040072""","""8557430""","""GARANTIA EMPRESARIAL""",1,"""BANCO MERCANTIL DEL NORTE""",null,2004-05-28 00:00:00,3003,3003,3500,259316.0,54,2003-07-14 00:00:00,5,"""M""","""SMU-950315-790""",-10000.0,0.0,null,50.0,3,"""RT""","""SILLAS Y MUEBLES SA""",2003-08-14 18:42:03,19.3957,1545.0,"""Paraguas""","""PP""","""Tradicional""",3.27215,0,"""R""","""SG""","""CF""",193957.0,-0.0,null,193957.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""6499105748249""",2025-05-31 00:00:00,"""64991057""","""48249""","""GARANTIAS BANCOMEXT""",1,"""BANCOMEXT/BAJIO""",2025-01-03 14:07:17,2025-01-08 00:00:00,39336,3999,39336,7.3826e6,54,2022-05-27 00:00:00,4,"""M""","""PAU-130307-UM1""",-240938.06,-6340.27,0.0,90.0,10,null,"""PYA AUTOMOTIVE S DE RL DE CV""",2022-06-08 09:08:29,19.3957,1225.0,"""Bancomext""","""PP""","""Tradicional""",7.33128,1,"""NR""","""SG""","""CF""",4.6732e6,122973.974839,-0.0,4.7961e6
"""6499105748246""",2025-05-31 00:00:00,"""64991057""","""48246""","""GARANTIAS BANCOMEXT""",1,"""BANCOMEXT/BAJIO""",2025-01-16 13:38:10,2025-01-22 00:00:00,39336,3999,39336,2.4176e6,54,2022-05-27 00:00:00,4,"""M""","""PAU-130307-UM1""",-77032.59,-1905.18,0.0,90.0,10,null,"""PYA AUTOMOTIVE S DE RL DE CV""",20

### Querie Recuperaciones

#### Ejecución

In [7]:
def main_querie_recuperaciones():
    dwh_recuperaciones = importa_recuperaciones()
    recuperadas_global_inter = genera_recuperadas_global_inter(dwh_recuperaciones)
    recuperadas_global_vf = genera_recuperadas_global_vf(recuperadas_global_inter)

    #recuperadas_global_vf_bancomext = genera_recuperadas_global_bancomext(recuperadas_global_vf)
    #recuperadas_global_vf_sin_bancomext = genera_recuperadas_global_sin_bancomext(recuperadas_global_vf)

    valida_dwh_dac = genera_valida_dwh_dac(dwh_recuperaciones)
    valida_td = genera_valida_td(dwh_recuperaciones)

    # Exporta Tablas
    # recuperadas_global_vf.write_csv(fl_recuperadas_global_vf, datetime_format="%Y-%m-%d", float_precision=6)
    recuperadas_global_vf.write_parquet(fl_recuperadas_global_vf)

    # Exporta Validadores
    valida_dwh_dac.write_csv(fl_valida_dwh_dac, float_precision=10, float_scientific=False)
    valida_td.write_csv(fl_valida_td,float_precision=10, float_scientific=False)
    
    return recuperadas_global_vf

recuperadas_global_vf = main_querie_recuperaciones()

### Querie Union Flujos

#### Ejecución

In [8]:
def main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf):
    pagos_agrup = genera_pagos_agrup(pagadas_global_vf)
    pagos_agrup = genera_concatenado(pagos_agrup, intermediario="Intermediario_Id", no_credito="Numero_Credito")

    recuperadas_global_vf = genera_concatenado(recuperadas_global_vf)
    #recuperaciones_agrup = genera_recuperaciones_agrup(recuperadas_global_vf)

    # Falta depurar columnas de B
    #uf_pagos_recuperaciones = pagos_agrup.join(recuperadas_global_vf, on="Concatenado", how='left')

    # Falta depurar columnas de B
    uf_recuperaciones_pagos = recuperadas_global_vf.join(pagos_agrup, on="Concatenado", how='left')

    #t_roberto = pagos_agrup.join(recuperaciones_agrup, on="Concatenado", how='left')

    recupera_con_pagos_flujos = genera_recupera_con_pagos_flujos(uf_recuperaciones_pagos)
    #recupera_con_pagos_flujos_bmxt = genera_recupera_con_pagos_flujos_bancomext(recupera_con_pagos_flujos)
    #recupera_con_pagos_flujos_sin_bmxt = genera_recupera_con_pagos_flujos_sin_bancomext(recupera_con_pagos_flujos)

    valida_base_pagos_recup = genera_valida_base_pagos_recup(recuperadas_global_vf)
    valida_base_pagos_recup_mn = genera_valida_base_pagos_recup_mn(recuperadas_global_vf)

    # Exporta tablas
    # recupera_con_pagos_flujos.write_csv(fl_recupera_con_pagos_flujos, datetime_format="%Y-%m-%d", float_precision=6)
    recupera_con_pagos_flujos.write_parquet(fl_recupera_con_pagos_flujos)

    # Exporta Validadores
    valida_base_pagos_recup.write_csv(fl_valida_base_recup)
    valida_base_pagos_recup_mn.write_csv(fl_valida_base_recup_mn)

    return 0

main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)


0

In [9]:
pagadas_global_vf['Monto_Pagado_Mn'].sum()

66063669834.664955

### Ejecución Principal

In [10]:
# if __name__ ==  "__main__":
#     pagadas_global_vf = main_querie_pagadas()
#     recuperadas_global_vf = main_querie_recuperaciones()
#     main_querie_union_flujos(pagadas_global_vf, recuperadas_global_vf)

In [11]:
end_time = time.time()
end_time - start_time

25.949511528015137